In [1]:
cd ../..

/home/paperspace/time-varying-graphs


In [2]:
import numpy as np
from omegaconf import OmegaConf
from sklearn.metrics import *
from tqdm.auto import tqdm

import stereo
from stereo.utils.results import *

npi = np.intersect1d
npc = np.concatenate
npu = np.unique
npd = np.setdiff1d

In [3]:
model_name = 'Autoreg'
dataset_name = 'PBMC'

In [4]:
cfg = OmegaConf.load(f'configs/dataset/GRN-{dataset_name}.yaml')
dataset = load_dataset(cfg)
training_tfs = dataset.genes[dataset.gene_is_tf]

Keeping 2050 genes.


### Run seeds and IDs

In [5]:
runs = load_run_ids(f'results/runs/PBMC/{model_name}.json', 'DEG4')

In [6]:
As = []

for seed, run in tqdm(runs.items()):
    ckpt_path = f'PBMCGeneRegPseudotimeDataset_logs/{run}/checkpoints/last.ckpt'
    module = stereo.TemporalDeepAutoreg_Module.load_from_checkpoint(ckpt_path, map_location='cpu')
    As.append([A.detach().numpy() for A in module.A_seq_])

As = np.stack(As)
print(As.shape)

  0%|          | 0/10 [00:00<?, ?it/s]

(10, 3, 2050, 590)


In [13]:
t_to_selected = aggregate_As(
    As,
    tfs=dataset.genes[module.sources_mask_seq[0]],
    genes=dataset.genes,
    consensus=6,
    top_tfs=38,
)

t=0	 Selected 1166 edges	N. TFs = 38	N. genes = 924	
t=1	 Selected 1017 edges	N. TFs = 38	N. genes = 862	
t=2	 Selected 728 edges	N. TFs = 38	N. genes = 659	


In [15]:
write_graphs(t_to_selected, f'results/graphs/{model_name}-{dataset_name}')